# Forecasting Stock Prices
**Data Analytics and Big Data Capstone Project 2018c-5 - Data Science with Python**

### 4. Finding Best Model

In [ ]:
# Load libraries
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns; sns.set()

from matplotlib.pyplot import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

# plot within notebook
%matplotlib inline

warnings.filterwarnings('ignore')

In [ ]:
## load parameters

# set figure size
rcParams['figure.figsize'] = 15, 8

In [ ]:
# Load dataset
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')

# load data
df_adj = pd.read_csv('.\\data\\eod.data-intc-adjusted.lv1.csv')

In [ ]:
# setting the index as date
df_adj['date'] = pd.to_datetime(df_adj.date, format='%Y-%m-%d')
df_adj.index = df_adj['date']
df_adj.drop(['date'], axis=1, inplace=True)
df_adj.sort_index(ascending=True, axis=0, inplace=True)
df_adj.head(5)